In [2]:
import pandas as pd
import geopandas as gpd
import sys
import matplotlib.pyplot as plt 
import seaborn as sns
import rasterio as rio
from rasterio.plot import plotting_extent
import earthpy.plot as ep
import numpy as np
from rasterstats import zonal_stats
from shapely import wkt
dir_path = './data/raw/'

In [6]:
%%time

# Building footprints
buildings = gpd.read_file('./data/raw/building_footprints_google_w_districts.geojson')
# to lowercase
buildings.District = buildings.District.str.lower()
buildings['origin_origin_id'] = buildings.origin +'_'+buildings.origin_id # Identifier
buildings_geom = buildings[['origin_origin_id','geometry']].copy()

# Use projected CRS
crs = 'epsg:32735'
buildings_geom_utm_RW = buildings_geom.to_crs(crs)

# meter data
meter_data = pd.read_pickle('./data/processed/reliabilty_cons_w_conn_days_22_11_22_12am.pck')
meter_building_dist = 80 # max distance between meter and building
# Match building footprints to nearest meter locations
meter_data_w_bf = gpd.sjoin_nearest(meter_data, buildings_geom_utm_RW,
                                    how='left', distance_col="meter_building_distance", max_distance = meter_building_dist).drop('index_right',axis=1)
meter_data_w_bf_w_consumption = meter_data_w_bf[meter_data_w_bf.annual_consumption_2019.notna()] # remove buildings with no recorded consumption
buildings_w_consumption_lst = meter_data_w_bf_w_consumption.origin_origin_id.unique().tolist()
buildings_geom_w_consumption = buildings_geom_utm_RW[buildings_geom_utm_RW.origin_origin_id.isin(buildings_w_consumption_lst)].copy() # buildings with consumption





CPU times: user 11min 9s, sys: 20.6 s, total: 11min 29s
Wall time: 11min 36s


# ADD Raster features

In [3]:
%%time
# load building features
building_features = pd.read_pickle('./data/processed/buildings_features_22_11_22_4pm.pck') # load building features from vectors

# convert to village features
village_features = building_features.groupby('village_id').mean()

## Import raster features
# import landcover files
df_lc_2015 = pd.read_pickle(dir_path+'lc_2015.pck')
df_lc_2010 = pd.read_pickle(dir_path+'lc_2010.pck')
df_lc_2000 = pd.read_pickle(dir_path+'lc_2000.pck')
df_lc_1990 = pd.read_pickle(dir_path+'lc_1990.pck')
# import population files
df_popn_2005 = pd.read_pickle(dir_path+'village_mean_popn_2005.pck')
df_popn_2015 = pd.read_pickle(dir_path+'village_mean_popn_2015.pck')
# import elevation file
df_elev = pd.read_pickle(dir_path+'village_mean_elevation.pck')
# import AWI
df_AWI_2016 = pd.read_pickle(dir_path+'AWI_2016.pck')
df_AWI_2017 = pd.read_pickle(dir_path+'AWI_2017.pck')
df_AWI_2018 = pd.read_pickle(dir_path+'AWI_2018.pck')
df_AWI_2019 = pd.read_pickle(dir_path+'AWI_2019.pck')
# import spending
df_spending_2016 = pd.read_pickle(dir_path+'spending_2016.pck')
df_spending_2017 = pd.read_pickle(dir_path+'spending_2017.pck')
df_spending_2018 = pd.read_pickle(dir_path+'spending_2018.pck')
df_spending_2019 = pd.read_pickle(dir_path+'spending_2019.pck')

CPU times: user 5.27 s, sys: 3 s, total: 8.27 s
Wall time: 9.47 s


In [4]:
%%time
# merge vector and raster features

# LULC
village_features = village_features.merge(df_lc_2015[['village_id','majority']], how='left',on='village_id').rename(columns={'majority':'majority_2015'})
village_features = village_features.merge(df_lc_2010[['village_id','majority']], how='left',on='village_id').rename(columns={'majority':'majority_2010'})
village_features = village_features.merge(df_lc_2000[['village_id','majority']], how='left',on='village_id').rename(columns={'majority':'majority_2000'})
village_features = village_features.merge(df_lc_1990[['village_id','majority']], how='left',on='village_id').rename(columns={'majority':'majority_1990'})
# Popn
village_features = village_features.merge(df_popn_2005[['village_id','mean_popn']], how='left',on='village_id').rename(columns={'mean_popn':'mean_popn_2005'})
village_features = village_features.merge(df_popn_2015[['village_id','mean_popn']], how='left',on='village_id').rename(columns={'mean_popn':'mean_popn_2015'})
village_features = village_features.merge(df_elev[['village_id','mean_elevation']], how='left',on='village_id')
# AWI
village_features = village_features.merge(df_AWI_2016[['village_id','mean_AWI']], how='left',on='village_id').rename(columns={'mean_AWI':'mean_AWI_2016'})
village_features = village_features.merge(df_AWI_2017[['village_id','mean_AWI']], how='left',on='village_id').rename(columns={'mean_AWI':'mean_AWI_2017'})
village_features = village_features.merge(df_AWI_2018[['village_id','mean_AWI']], how='left',on='village_id').rename(columns={'mean_AWI':'mean_AWI_2018'})
village_features = village_features.merge(df_AWI_2019[['village_id','mean_AWI']], how='left',on='village_id').rename(columns={'mean_AWI':'mean_AWI_2019'})
# spending
village_features = village_features.merge(df_spending_2016[['village_id','mean_spending']], how='left',on='village_id').rename(columns={'mean_spending':'mean_spending_2016'})
village_features = village_features.merge(df_spending_2017[['village_id','mean_spending']], how='left',on='village_id').rename(columns={'mean_spending':'mean_spending_2017'})
village_features = village_features.merge(df_spending_2018[['village_id','mean_spending']], how='left',on='village_id').rename(columns={'mean_spending':'mean_spending_2018'})
village_features = village_features.merge(df_spending_2019[['village_id','mean_spending']], how='left',on='village_id').rename(columns={'mean_spending':'mean_spending_2019'})

# village density
    # all buildings
village_density = building_features.groupby(['village_id'])['origin_origin_id'].count().reset_index().rename(columns={'origin_origin_id':'village_density_all'})
village_features = village_features.merge(village_density, how='left',on='village_id')
   


CPU times: user 548 ms, sys: 56.6 ms, total: 604 ms
Wall time: 605 ms


In [30]:
%%time


def filter_categories(list_items):
    '''
    function to select unique categories for each building
    If building has one non-residential categorize it as non residential
    '''
    item = ''
    if len(list_items)>1:
        
        for item in list_items:
            if item=='Non Residential':
                item = 'Non Residential'
            else:
                item= 'Residential'
        return item
    else:
        return list_items[0] 

#unique categories for each building
meter_data_cats = meter_data_w_bf_w_consumption.groupby(['origin_origin_id'])['eguide_categories'].unique().reset_index()
meter_data_cats['eguide_categories'] = meter_data_cats['eguide_categories'].apply(lambda x: filter_categories(x))

    
# Building 2019 total consumption
buiding_kWh = meter_data_w_bf_w_consumption.groupby(['origin_origin_id'])['annual_consumption_2019'].sum(min_count=1).reset_index()
# Building days since connection, saifi, saidi
meter_data_features =meter_data_w_bf_w_consumption.groupby(['origin_origin_id'])[['days_since_connection','saifi_2020','saifi_2019','saifi_2018','saifi_2017','saifi_2016',
                                                                                 'saidi_2020','saidi_2019','saidi_2018','saidi_2017','saidi_2016']].max().reset_index()

# # merge meter features
# meter_data_features = meter_data_features.merge(buiding_kWh, how='left',on='origin_origin_id')
# meter_data_features = meter_data_features.merge(meter_data_cats, how='left',on='origin_origin_id')

# merge meter and village id
meter_data_features = meter_data_features.merge(building_features[['village_id','origin_origin_id']], on='origin_origin_id', how='left')
buiding_kWh = buiding_kWh.merge(building_features[['village_id','origin_origin_id']], on='origin_origin_id', how='left')
meter_data_cats = meter_data_cats.merge(building_features[['village_id','origin_origin_id']], on='origin_origin_id', how='left')
meter_data_cats['eguide_categories']=meter_data_cats['eguide_categories'].apply(lambda x: 'Non Residential' if x=='other' else x) # meter categories

# Groupby villages
meter_data_cats = meter_data_cats.groupby(['village_id','eguide_categories'])[['eguide_categories','origin_origin_id']].count(
).rename(columns={'eguide_categories':'categories'}).reset_index().drop('origin_origin_id', axis=1).pivot(
    index='village_id',columns='eguide_categories',values='categories').reset_index().fillna(0)
buiding_kWh = buiding_kWh.groupby('village_id').mean().reset_index()
meter_data_features = meter_data_features.groupby('village_id').mean().reset_index()

# merge village meter features
meter_features = meter_data_features.merge(meter_data_cats, how='inner',on='village_id')
village_meter_features = village_features.merge(meter_features, how='inner',on='village_id')
village_meter_features = village_meter_features.merge(buiding_kWh, how='inner',on='village_id')
# meter_stats = meter_data_features.groupby('village_id').mean().reset_index()
# village_meter_features = village_cats.merge(meter_stats, how='inner', on='village_id')

# # merge village features with meter features
# village_features = village_features.merge(village_meter_features, how='left', on='village_id')

# # village level dataset
data_villages = village_meter_features[village_meter_features.annual_consumption_2019.notna()]
data_villages['Residential_percent'] =data_villages['Residential']/(data_villages['Non Residential']+data_villages['Residential'])
data_villages['non_Residential_percent'] =data_villages['Non Residential']/(data_villages['Non Residential']+data_villages['Residential'])

CPU times: user 31.5 s, sys: 940 ms, total: 32.5 s
Wall time: 32.5 s


In [31]:
# residential and non residential percentages
data_villages['Residential_percent'] =data_villages['Residential']/(data_villages['Non Residential']+data_villages['Residential'])
data_villages['non_Residential_percent'] =data_villages['Non Residential']/(data_villages['Non Residential']+data_villages['Residential'])

In [32]:
village_boundaries = gpd.read_file('./data/raw/Villages_boundaries.geojson').rename(columns={'code_vill_':'village_id'})
village_boundaries['district'] = village_boundaries['district'].str.lower()

# merge
data_villages = data_villages.merge(village_boundaries[['village_id','district','province']], how='left',on='village_id')

# remove NaNs
columns_w_nans_dist = ['mean_AWI_2019','mean_AWI_2018','mean_AWI_2017','mean_AWI_2016','mean_spending_2019','mean_spending_2018','mean_spending_2017','mean_spending_2016',
                  'saifi_2020','saifi_2019','saifi_2018','saifi_2017','saifi_2016','saidi_2020','saidi_2019','saidi_2018','saidi_2017','saidi_2016',
                 ]
for col in columns_w_nans_dist:
    data_villages[col] = data_villages.groupby("district")[col].transform(lambda x: x.fillna(x.mean())) # use distict average
    
columns_w_nans_prov = [
                  'saifi_2020','saifi_2019','saifi_2018','saifi_2017','saifi_2016','saidi_2020','saidi_2019','saidi_2018','saidi_2017','saidi_2016',
                 ]
for col in columns_w_nans_prov:
    data_villages[col] = data_villages.groupby("province")[col].transform(lambda x: x.fillna(x.mean())) # use province average
data_villages = data_villages.drop(['village_id','district','province'],axis=1)

village_id                             0
area_in_meters                         0
n_bldgs_1km_away                       0
school_building_dist                   0
health_building_dist                   0
trade_building_dist                    0
coffee_building_dist                   0
tourism_building_dist                  0
industrypark_building_dist             0
District_offices_building_dist         0
religious_centres_building_dist        0
sports_centres_building_dist           0
major_towns_building_dist              0
bus_stations_building_dist             0
landmark_building_dist                 0
road_netwk_building_dist               0
national_paved_road_building_dist      0
national_unpaved_road_building_dist    0
area_km                                0
majority_2015                          0
majority_2010                          0
majority_2000                          0
majority_1990                          0
mean_popn_2005                         0
mean_popn_2015  

### Save Dataframe

In [34]:
pck_fp = './data/processed/data_Villages_1_12_22_3pm_cat_percent_mean.pck'
data_villages.to_pickle(pck_fp)